In [53]:
import sys, os, re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
scrapytoolv2_path = os.path.join('..', 'scrapy_toolv2')
sys.path.append(scrapytoolv2_path)
from html_downloader import html_downloader as hd

In [63]:
class sappfrt():
    def __init__(self, path, first_index_url):
        self.first_index_url = first_index_url
        self.hd = hd(world=True)
        self.path = os.path.join('data',path)
        if not os.path.exists(self.path):
            os.makedirs(self.path)
    def parse_index_page(self, index_url, page_url=[]):
        r = self.hd.request_proxy(index_url)
        soup = BeautifulSoup(r.content,'html.parser')
        content_div = soup.find('div', {'class':'jar2l_list'})
        lis = content_div.find_all('li')
        for l in lis:
            tmp_url = l.a.get('href')
            tmp_url = urljoin(index_url, tmp_url)
            tmp_title = l.a.get_text()
            page_url.append([tmp_url, tmp_title])
        
        pages = soup.find('div', {'class':'page'})
        if pages:
            page_boxes = pages.find_all('a', {'class':'m2page_a'})
            for p in page_boxes:
                if p.get_text() == '>':
                    next_page = p
            next_page_url = next_page.get('href')
            if '.shtml' in next_page_url:
                next_page_url = urljoin(index_url, next_page_url)
                if next_page_url != index_url:
                    self.parse_index_page(next_page_url, page_url=page_url)
        return page_url
    def parse_content_page(self, page_url):
        r = self.hd.request_proxy(page_url[0])
        soup = BeautifulSoup(r.content, 'html.parser')
        table_div = soup.find('div', {'id':'artibody'})
        target_table = table_div.find('table')
        trs = target_table.find_all('tr')
        data = {}
        for i in range(0,len(trs)):
            if i:
                columns = list(data.keys())
                tds = trs[i].find_all('td')
                for j in range(0,len(tds)):
                    data[columns[j]].append(re.sub('\s','',tds[j].get_text())) 
            else:
                ths = trs[i].find_all('td')
                for j in ths:
                    data[re.sub('\s','',j.get_text())] = []
        df = pd.DataFrame(data)
        df['source_url'] = page_url[0]
        df['source'] = page_url[1]
        df.to_csv(os.path.join(self.path, '{}.csv'.format(page_url[1])), index=False)
        return df
    def main(self):
        df = pd.DataFrame()
        page_url = self.parse_index_page(self.first_index_url)
        for p in page_url:
            tmpdf = self.parse_content_page(p)
            df = df.append(tmpdf)
        df.to_csv(os.path.join(self.path, 'total.csv'), index = False)

In [11]:
change = 'http://www.sapprft.gov.cn/sapprft/channels/11083.shtml'
homemade = 'http://www.sapprft.gov.cn/sapprft/channels/7029.shtml'
import_net = 'http://www.sapprft.gov.cn/sapprft/channels/7027.shtml'
import_digital = 'http://www.sapprft.gov.cn/sapprft/channels/7028.shtml'

In [65]:
test1 = sappfrt('change', change)
test1.main()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [67]:
test2 = sappfrt('homemade',homemade)
test2.main()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [69]:
test3 = sappfrt('import_net', import_net)
test3.main()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [71]:
test4 = sappfrt('import_digital', import_digital)
test4.main()